In [1]:
import json
from config import settings
import os

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

import polars as pl
from polars import col as c
from polars import selectors as cs
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp
from distflow_algorithm import DistFlow
import time
import pandapower as pp
import pandapower.networks as pn
from polars_function import (
    get_transfo_admittance,
    get_transfo_impedance,
    get_transfo_conductance,
    get_transfo_imaginary_component,
)

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import get_all_edge_data, generate_shortest_path_length_matrix
import graphblas as gb
from utility.parser_utility import duckdb_to_changes_schema

## Import Schema from duckdb

In [2]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
changes_schema = duckdb_to_changes_schema(file_path=file_names["duckdb_output"])
list(changes_schema.__dict__.keys())

Read and validate tables from matlab_grid.db file: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.11it/s]


['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

In [13]:
changes_schema.resource.head(1)

diff,uuid,start_heartbeat,end_heartbeat,start,end,dso_code,concrete_class,name,feeder_fk,metadata,owner
str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,str
"""+""","""b4fdf239-cfe2-5453-91c3-84c1b2…",2025-01-22 13:55:58.105889 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_1""","""branch""",null,null,null,null


## Filter data

In [43]:
changes_schema.branch_parameter_event

diff,uuid,timestamp,heartbeat,source_fk,eq_fk,r,r0,x,x0,b,b0,g,g0
str,str,"datetime[μs, UTC]","datetime[μs, UTC]",str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""+""","""d5ac0574-b2fb-4cba-9ced-c56885…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""b4fdf239-cfe2-5453-91c3-84c1b2…",0.0031,0.0,0.0018075,0.0,0.000005,0.0,0.0,0.0
"""+""","""d930f739-8936-4836-82b9-cbc132…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""3edce6fc-e164-5b3e-9b0d-8a1c86…",0.03275,0.0,0.01834,0.0,0.000056,0.0,0.0,0.0
"""+""","""30f59e6d-4a37-4702-b74c-25f2c2…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""c173d7f3-f7ca-5f44-98d8-63c1aa…",0.04848,0.0,0.0014576,0.0,0.000001,0.0,0.0,0.0
"""+""","""1c3f510b-8166-4cd4-89da-393408…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""c33afba9-0379-5d6a-9ca1-743cf7…",0.00124,0.0,0.000723,0.0,0.000002,0.0,0.0,0.0
"""+""","""18fced87-2148-4a11-b9a9-6f3c5c…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""020bd95c-9320-5560-9627-4bbe19…",0.006625,0.0,0.00371,0.0,0.000011,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""48e0c9c2-3118-4719-a095-51c6d7…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""d9975d35-54c3-5993-b044-835e1b…",0.0606,0.0,0.001822,0.0,0.000002,0.0,0.0,0.0
"""+""","""a2ada99c-7ace-4a41-9cc5-95063c…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""e46fa9df-d633-5149-96b4-c9cccc…",0.001364,0.0,0.0007953,0.0,0.000002,0.0,0.0,0.0
"""+""","""54ef66d7-bca9-48d1-ab34-bddfc5…",1960-01-01 00:00:00 UTC,2025-01-22 13:55:58.105889 UTC,"""scada""","""34edd04e-a827-5cf8-b233-43d285…",0.000868,0.0,0.0005061,0.0,0.000001,0.0,0.0,0.0


switch

branch

branch_parameter_event

In [50]:
line_data = changes_schema.resource.filter(
    (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
).select(c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type"))

cn_mapping = pl_to_dict(
    changes_schema.connectivity.select(
        pl.concat_str("eq_fk", "side").alias("id"), "uuid"
    )
)
## Add uuid of t1 and t2 for resource
line_data = line_data.with_columns(
    (c("uuid") + "t1")
    .replace_strict(cn_mapping, default=None)
    .alias("u_of_edge"),  # Replace eq_fk with uuid from connectivity for branch
    (c("uuid") + "t2").replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)

# Add branch parameter
branch = changes_schema.branch_parameter_event.select(
    c("uuid"),
    # c("source_fk"),
    c("eq_fk"),
    c("r"),
    c("r0"),
    c("x"),
    c("x0"),
    c("b"),
    c("b0"),
    c("g"),
    c("g0"),
)
# Add branch parameter to line_data
line_data = line_data.join(branch, left_on="uuid", right_on="eq_fk", how="left").drop(
    "uuid_right"
)

In [30]:
# ## Work but don't take the switch into account, make it with connectivity_node at first
# branch = changes_schema.branch_parameter_event.select(
#     c("uuid"),
#     # c("source_fk"),
#     c("eq_fk"),
#     c("r"),
#     c("r0"),
#     c("x"),
#     c("x0"),
#     c("b"),
#     c("b0"),
#     c("g"),
#     c("g0"),
# )

# cn_mapping = pl_to_dict(
#     changes_schema.connectivity.select(
#         pl.concat_str("eq_fk", "side").alias("id"), "uuid"
#     )
# )
# line_data_2 = branch.with_columns(
#     (c("eq_fk") + "t1")
#     .replace_strict(cn_mapping, default=None)
#     .alias("u_of_edge"),  # Replace eq_fk with uuid from connectivity for branch
#     (c("eq_fk") + "t2").replace_strict(cn_mapping, default=None).alias("v_of_edge"),
# )

# line_id_mapping = pl_to_dict(
#     changes_schema.resource.filter(
#         (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
#     ).select("uuid", "dso_code")
# )

# line_type_mapping = pl_to_dict(
#     changes_schema.resource.filter(
#         (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
#     ).select("uuid", "concrete_class")
# )
# line_data_2 = branch.with_columns(
#     c("eq_fk")
#     .replace_strict(line_id_mapping, default=None)
#     .alias("line_id"),  # add dso_code to line_id
#     c("eq_fk")
#     .replace_strict(line_type_mapping, default=None)
#     .alias("type"),  # add dso_code to line_id
# )

# line_data_2
# # replace with the node id for u_of_edge and v_of_edge

uuid,eq_fk,r,r0,x,x0,b,b0,g,g0,line_id,type
str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str
"""d5ac0574-b2fb-4cba-9ced-c56885…","""b4fdf239-cfe2-5453-91c3-84c1b2…",0.0031,0.0,0.0018075,0.0,0.000005,0.0,0.0,0.0,"""line_1""","""branch"""
"""d930f739-8936-4836-82b9-cbc132…","""3edce6fc-e164-5b3e-9b0d-8a1c86…",0.03275,0.0,0.01834,0.0,0.000056,0.0,0.0,0.0,"""line_2""","""branch"""
"""30f59e6d-4a37-4702-b74c-25f2c2…","""c173d7f3-f7ca-5f44-98d8-63c1aa…",0.04848,0.0,0.0014576,0.0,0.000001,0.0,0.0,0.0,"""line_4""","""branch"""
"""1c3f510b-8166-4cd4-89da-393408…","""c33afba9-0379-5d6a-9ca1-743cf7…",0.00124,0.0,0.000723,0.0,0.000002,0.0,0.0,0.0,"""line_5""","""branch"""
"""18fced87-2148-4a11-b9a9-6f3c5c…","""020bd95c-9320-5560-9627-4bbe19…",0.006625,0.0,0.00371,0.0,0.000011,0.0,0.0,0.0,"""line_6""","""branch"""
…,…,…,…,…,…,…,…,…,…,…,…
"""48e0c9c2-3118-4719-a095-51c6d7…","""d9975d35-54c3-5993-b044-835e1b…",0.0606,0.0,0.001822,0.0,0.000002,0.0,0.0,0.0,"""line_53""","""branch"""
"""a2ada99c-7ace-4a41-9cc5-95063c…","""e46fa9df-d633-5149-96b4-c9cccc…",0.001364,0.0,0.0007953,0.0,0.000002,0.0,0.0,0.0,"""line_54""","""branch"""
"""54ef66d7-bca9-48d1-ab34-bddfc5…","""34edd04e-a827-5cf8-b233-43d285…",0.000868,0.0,0.0005061,0.0,0.000001,0.0,0.0,0.0,"""line_55""","""branch"""


In [5]:
changes_schema.connectivity.filter(c("eq_fk") == "b4fdf239-cfe2-5453-91c3-84c1b271b8bc")

diff,uuid,start_heartbeat,end_heartbeat,start,end,eq_fk,side,eq_class,abstraction_fk,cn_fk,container_fk,indirect
str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,str,bool
"""+""","""699ac222-d2ec-49e1-bce6-c63186…",2025-01-22 13:55:58.105889 UTC,null,1960-01-01 00:00:00 UTC,null,"""b4fdf239-cfe2-5453-91c3-84c1b2…","""t1""","""branch""","""physical""","""ba84d70a-80d7-590e-b112-f9c4b5…",null,false
"""+""","""b95d6f3f-049c-4cd6-8434-0ef53a…",2025-01-22 13:55:58.105889 UTC,null,1960-01-01 00:00:00 UTC,null,"""b4fdf239-cfe2-5453-91c3-84c1b2…","""t2""","""branch""","""physical""","""df941fce-ceda-5874-ab63-5c8af9…",null,false


In [6]:
ext_grid_id = "1"
n = 0.95
v_ext_grid_sq = 1.05

line_data: pl.DataFrame = pl.DataFrame(
    {
        "line_id": np.arange(1, 13).astype(str),
        "type": ["branch"] * 10 + ["transformer"] * 2,
        "u_of_edge": ["2", "2", "2", "4", "5", "6", "7", "10", "12", "13", "8", "9"],
        "v_of_edge": ["11", "1", "3", "1", "4", "4", "8", "9", "11", "11", "4", "8"],
        "n_transfo": [1.0] * 10 + [n**2, n**2],
        "x_pu": np.arange(1, 13) * 2e-3,
        "r_pu": np.arange(1, 13) * 1e-3,
        "b_pu": list(np.arange(1, 11) * -1e-3) + [0.001, 0.001],
        "g_pu": [0.0] * 10 + [0.001, 0.001],
    }
)

node_data: pl.DataFrame = pl.DataFrame(
    {
        "node_id": np.arange(1, 14).astype(str),
        "v_base": [400.0] * 7 + [200.0, 400.0, 400.0, 200.0, 100.0, 100.0],
        "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10]) * 1e-2,
        "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1]) * 1e-2,
    },
    strict=False,
)

np.set_printoptions(linewidth=200)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq

s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 0.0006990432739257812
graphblas: 0.011247634887695312


In [7]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("../data/input_grid/modified_cigre_network_lv.p")
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

UserWarning: File ../data/input_grid/modified_cigre_network_lv.p does not exist!!

In [4]:
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow: DistFlow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [5]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 0.0007200241088867188
graphblas: 0.022676706314086914


In [84]:
distflow.h_gb.T

"M_150.T"            nvals  nrows  ncols  dtype         format
gb.TransposedMatrix     41     13     13   FP64  bitmapc (iso)
--------------------------------------------------------------
     0    1    2    3    4    5    6    7    8    9    10   11   12
0   1.0            1.0                      1.0  1.0  1.0          
1        1.0  1.0                                     1.0          
2             1.0                                     1.0          
3                  1.0                      1.0       1.0          
4                       1.0                 1.0       1.0          
5                  1.0       1.0            1.0       1.0          
6                                 1.0       1.0       1.0          
7        1.0  1.0                      1.0            1.0          
8                                           1.0       1.0          
9                  1.0                      1.0  1.0  1.0          
10                                                    1.0          
11            1.0                                     1.0  1.0     
12       1.0  1.0                                     1.0       1.0

In [82]:
np.multiply(s_node.reshape(-1, 1), np.random.random(100) + 0.5).transpose()

array([[0.        +0.j        , 0.95179472+0.j        , 0.09517947+0.j        , ..., 0.        +0.j        , 0.01903589+0.00190359j, 0.09517947+0.00951795j],
       [0.        +0.j        , 0.57482819+0.j        , 0.05748282+0.j        , ..., 0.        +0.j        , 0.01149656+0.00114966j, 0.05748282+0.00574828j],
       [0.        +0.j        , 0.83182895+0.j        , 0.08318289+0.j        , ..., 0.        +0.j        , 0.01663658+0.00166366j, 0.08318289+0.00831829j],
       ...,
       [0.        +0.j        , 1.41329948+0.j        , 0.14132995+0.j        , ..., 0.        +0.j        , 0.02826599+0.0028266j , 0.14132995+0.01413299j],
       [0.        +0.j        , 0.84920213+0.j        , 0.08492021+0.j        , ..., 0.        +0.j        , 0.01698404+0.0016984j , 0.08492021+0.00849202j],
       [0.        +0.j        , 0.82929242+0.j        , 0.08292924+0.j        , ..., 0.        +0.j        , 0.01658585+0.00165858j, 0.08292924+0.00829292j]])

In [77]:
s_node.reshape(-1, 1)

array([[ 0.   +0.j   ],
       [ 1.   +0.j   ],
       [ 0.1  +0.j   ],
       [ 0.07 +0.007j],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [-0.02 -0.002j],
       [ 0.005+0.005j],
       [ 0.   +0.02j ],
       [ 0.25 +0.j   ],
       [ 0.   +0.j   ],
       [ 0.02 +0.002j],
       [ 0.1  +0.01j ]])

In [73]:
np.random.random(100) + 0.5

array([1.20979763, 0.7342991 , 1.17402437, 0.7730132 , 1.4262539 , 0.79595615, 1.01598385, 1.1852591 , 0.61825153, 0.9891824 , 1.15020281, 1.36583277, 1.07364581, 1.09137555, 1.11333436, 0.83503185,
       0.50154774, 0.68246054, 1.20212403, 1.23478103, 0.53457727, 1.13480669, 1.19219885, 1.43527789, 1.28531631, 1.00463642, 0.58823296, 0.56234986, 1.34667091, 1.15340563, 0.5594287 , 1.23243907,
       1.07628486, 0.62812337, 1.08384573, 0.77446393, 1.19966667, 1.28676713, 0.87357647, 1.21993883, 1.38870329, 1.42080445, 1.1968925 , 0.60623033, 0.58885734, 1.35222399, 1.19792729, 1.47520514,
       0.50217659, 1.30814651, 1.05854168, 0.5674643 , 0.91687194, 1.14234268, 0.79472783, 1.1585023 , 0.95823956, 1.15504163, 1.00367084, 1.38570815, 0.68326989, 1.44255223, 0.94303185, 1.17076271,
       1.02690779, 1.37093334, 1.48342139, 0.60668063, 1.03957436, 0.81959101, 0.65890734, 1.28476738, 1.14761778, 1.22012232, 0.68574922, 1.12463729, 1.17966652, 1.19622725, 0.70527933, 1.25037436,
     

In [6]:
line_res_pp = pl.DataFrame(
    {
        "node_id": list(net.line["to_bus"].astype(str)),
        "p_pp": list(net.res_line["p_from_mw"]),
        "q_pp": list(net.res_line["q_from_mvar"]),
    }
)

node_res_pp = pl.from_pandas(net.res_bus["vm_pu"], include_index=True).select(
    c("index").cast(pl.Utf8).alias("node_id"), "vm_pu"
)

result = (
    pl.DataFrame(
        {
            "v": np.sqrt(v),
            "p": np.real(s_flow),
            "q": np.imag(s_flow),
        }
    )
    .with_row_index(name="node_id")
    .with_columns(
        c("node_id")
        .replace_strict(distflow.node_nb_to_id_mapping, default=None)
        .alias("node_id")
    )
    .join(line_res_pp, on="node_id", how="left")
    .join(node_res_pp, on="node_id", how="left")
    .with_columns(
        ((c("v") - c("vm_pu")) / c("vm_pu")).abs().alias("diff_v"),
        ((c("p") - c("p_pp")) / c("p_pp")).abs().alias("diff_p"),
        ((c("q") - c("q_pp")) / c("p_pp")).abs().alias("diff_q"),
    )
)
print(result["diff_v", "diff_p", "diff_q"].max())

shape: (1, 3)
┌──────────┬──────────┬──────────┐
│ diff_v   ┆ diff_p   ┆ diff_q   │
│ ---      ┆ ---      ┆ ---      │
│ f64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ 0.000392 ┆ 0.000182 ┆ 0.010306 │
└──────────┴──────────┴──────────┘


In [7]:
net: pp.pandapowerNet = pp.from_pickle("../data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 0.5958316326141357


In [8]:
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [10]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 5.96026873588562
graphblas: 0.0697178840637207


node_data

In [56]:
node_data

node_id,v_base,p_node_pu,q_node_pu
str,f64,f64,f64
"""0""",20000.0,0.0,0.0
"""1""",20000.0,0.0,0.0
"""2""",20000.0,0.0,0.0
"""3""",20000.0,0.0,0.0
"""4""",20000.0,0.0,0.0
…,…,…,…
"""9693""",20000.0,0.893208,-0.000019
"""9694""",20000.0,0.0,0.0
"""9695""",20000.0,0.0,0.0


In [58]:
line_data = line_data.group_by("u_of_edge", "v_of_edge").agg(
    c("name", "n_transfo", "type").first(),
    c("r_pu", "x_pu").first() / c("r_pu", "x_pu").count(),
    c("g_pu", "b_pu").first() * c("g_pu", "b_pu").count(),
)

In [60]:
nx_graph = nx.Graph()
_ = line_data.with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_graph)
)

nx.cycle_basis(nx_graph)

[['2403', '6792', '9322'],
 ['2837',
  '7146',
  '9368',
  '7147',
  '2838',
  '9322',
  '7140',
  '2832',
  '9366',
  '7145',
  '2836',
  '9367'],
 ['6093', '1601', '9244'],
 ['6928', '2566', '9338'],
 ['8537', '4392', '4391', '9439'],
 ['4245', '8388', '4241', '9511'],
 ['3922', '8111', '3923', '9395'],
 ['3427', '7653', '3426', '9395'],
 ['4998', '367', '9106'],
 ['8636', '4493', '9107'],
 ['8643', '4498', '9396'],
 ['1085', '5660', '9189'],
 ['4221', '8368', '9207'],
 ['7558',
  '3316',
  '3347',
  '9350',
  '7588',
  '3348',
  '9423',
  '8378',
  '4233',
  '4232',
  '9512',
  '8373',
  '4226',
  '9207',
  '5848',
  '1298',
  '1295',
  '1294',
  '3276',
  '9209'],
 ['8933', '4817', '9570', '4813', '8931', '9225', '2370', '6766', '9209'],
 ['7547', '3301', '9415'],
 ['957', '5549', '9170'],
 ['3726', '7927', '9447', '3725', '7926', '9118'],
 ['3',
  '193',
  '154',
  '9681',
  '64',
  '236',
  '65',
  '9682',
  '234',
  '61',
  '9678',
  '69',
  '155',
  '293',
  '9730',
  '253',
  